# Evaluation: Résumé de Texte

## Travail pratique#2 NLP#2

In [3]:
#Installation de packages
%pip install datasets
%pip install evaluate
%pip install scikit-learn
%pip install rouge_score
%pip install tensorflow
%pip install tf-keras
%pip install torch torchvision torchaudio
%pip install transformers
%pip install matplotlib

In [ ]:
# importation des librairies

import re
import pandas as pd
from sklearn.model_selection import  train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, BartForConditionalGeneration, BartTokenizer, BartForSequenceClassification, TFBartForSequenceClassification
from transformers import Trainer, TrainingArguments
from nltk.translate.bleu_score import sentence_bleu
import evaluate #to import metrics because the load_metric function has been removed from datasets library
from datasets import load_dataset, load_from_disk
import tensorflow as tf

## 1. Téléchargement et préparation du jeu de données

### Téléchargement et Pretraitement du jeu de données CNN/Daily Mail depuis Hugging Face Datasets.

In [4]:
# Téléchargement du jeu de données CNN/Daily Mail depuis Hugging Face Datasets
ds = load_dataset("cnn_dailymail", "3.0.0")

# Chargement des textes et de leurs résumés
texts = ds['train']['article']
summaries = ds['train']['highlights']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [5]:
# Prétraitement et Nettoyage du texte
def clean_text(text):
    # Suppression des caractères spéciaux et normalisation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # pour Supprumer des caractères spéciaux
    text = text.lower()  # Mettre le texte en minuscule
    return text

cleaned_texts = [clean_text(text) for text in texts]

In [6]:
# Division des données en train, validation et test sets (80/10/10)
X_train, X_temp, y_train, y_temp = train_test_split(cleaned_texts, summaries, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
total_size = len(X_train) + len(X_val) + len(X_test)
print(f"Training set size: {len(X_train)}"," soit: ",round((len(X_train)/total_size)*100,2),"%")
print(f"Validation set size: {len(X_val)}"," soit: ",round((len(X_val)/total_size)*100,2),"%")
print(f"Test set size: {len(X_test)}"," soit: ",round((len(X_test)/total_size)*100,2),"%")

Training set size: 229690  soit:  80.0 %
Validation set size: 28711  soit:  10.0 %
Test set size: 28712  soit:  10.0 %


In [ ]:
#histogramme de X_train X_val X_test

plt.hist([len(text.split()) for text in X_train], bins=50, alpha=0.5, label='Train')
plt.hist([len(text.split()) for text in X_val], bins=50, alpha=0.5, label='Validation')
plt.hist([len(text.split()) for text in X_test], bins=50, alpha=0.5, label='Test')
plt.legend(loc='upper right')
plt.title('Taille des phrases dans les ensembles de données')

## 2. Modèle de résumé extractif (Utiliser le jeu de données de tests uniquement)

In [7]:
# Charger le tokenizer et le modèle BERT
tokenizer_Bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_Bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#Preparation des donnees
def encode_data(texts, summaries):
    # Tokenisation des textes
    inputs = tokenizer_Bert(texts, padding=True, truncation=True, return_tensors="tf", max_length=512)
        
    # Assurez-vous que summaries est une liste de chaînes
    if isinstance(summaries, tf.Tensor):
        summaries = summaries.numpy().tolist()  # Convertir en liste si c'est un tensor

    # Vérifiez que les résumés sont bien des chaînes
    if not all(isinstance(summary, str) for summary in summaries):
        raise ValueError("Tous les éléments de 'summaries' doivent être des chaînes.")

    # Créer des labels : 1 si la phrase est dans le résumé, 0 sinon
    labels = []
    for sentence in texts:
    # Vérifiez si la phrase (sentence) est présente dans l'une des résumés
        if any(sentence in summary for summary in summaries):
            labels.append(1)  # Phrase clé
        else:
            labels.append(0)  # Phrase non clé

    return inputs['input_ids'], inputs['attention_mask'], tf.convert_to_tensor(labels)
    

In [9]:
#Extraction de phrases cles

def extract_key_sentences(text, num_sentences=3):
    # Tokeniser le texte en phrases
    sentences = text.split('. ')

    # Préparer les entrées pour le modèle
    inputs = tokenizer_Bert(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Prédire avec le modèle
    with torch.no_grad():
        logits = model_Bert(inputs['input_ids'], attention_mask=inputs['attention_mask']).logits
        probabilities = torch.softmax(logits, dim=1)[:, 1]  # Probabilité d'inclusion

    # Extraire les phrases avec les plus hautes probabilités
    sentence_indices = probabilities.argsort(descending=True)[:num_sentences]
    key_sentences = [sentences[i] for i in sentence_indices]
    return key_sentences

In [10]:
#Transformons la liste de chaine de caracteres X_test en texte brute et extraction des prhases cles
X_test_totext = " ".join(X_test)
key_sentences = extract_key_sentences(X_test_totext)
print("Phrases clés extraites : ", key_sentences[:5])

Phrases clés extraites :  ['Ceci est un exemple de texte', "Certaines sont plus importantes que d'autres", 'Il contient plusieurs phrases']


In [11]:
# Résumé généré par le modèle
resume_genere = "Ceci est un exemple de résumé généré."
# Résumé de référence
reference1 = ["Ceci est un exemple de résumé.", "Ceci est un résumé généré en exemple."]

# Calculer le score BLEU
bleu_score = sentence_bleu([ref.split() for ref in reference1], resume_genere.split())
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.6435


In [12]:
# Chargement du métrique ROUGE
rouge_metric = evaluate.load('rouge') #load_metric("rouge")

# Résumé généré par le modèle
resume_genere2 = "Ceci est un exemple de résumé généré."
# Résumé de référence
reference2 = ["Ceci est un exemple de résumé.", "Ceci est un résumé généré en exemple."]

predictions = [resume_genere2]
references = [[reference2]]  # Wrap the list of references in another list

# Calcul du score ROUGE
results = rouge_metric.compute(predictions=predictions, references=references)
#Affichage score
print(f"ROUGE-1 Score: {results['rouge1']:.4f}")
print(f"ROUGE-2 Score: {results['rouge2']:.4f}")
print(f"ROUGE-L Score: {results['rougeL']:.4f}")


ROUGE-1 Score: 0.7407
ROUGE-2 Score: 0.7200
ROUGE-L Score: 0.7407


## 3. Modèle de résumé abstrait (Encoder-Decoder)

Architecture :

Utiliser une architecture Transformer (BART ou T5).

Entraînement du modèle :

Utiliser les textes comme entrées pour l’encodeur et les résumés comme cibles pour le décodeur.
Définir la fonction de perte et une méthode d’optimisation adaptées.
Entraîner le modèle sur le jeu d'entraînement et valider sur le jeu de validation.


In [13]:
# Chargement du tokenizer et du modèle BART
tokenizer_Bart = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model_Bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [13]:
# Configuration de TensorFlow pour utiliser seulement une partie de la mémoire GPU (4Go) pour fqciliter l'entrainement du modele
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limiter l'utilisation de la mémoire GPU à 4 Go
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])  # Ajustez selon vos besoins
        print("Found GPU unit :",gpus[0])
    except RuntimeError as e:
        print(e)  # Les dispositifs virtuels doivent être configurés avant que le GPU ne soit initialisé

Found GPU at: /device:GPU:0


In [14]:
#Fonction de tokenisation
def tokenize_data(sentences):
    inputs = sentences['article']
    targets = sentences['highlights']
    model_inputs = tokenizer_Bart(inputs, max_length=1024, truncation=True)

    # Tokeniser le résumé
    with tokenizer_Bart.as_target_tokenizer():
        labels = tokenizer_Bart(targets, max_length=150, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
#Encodage de donnees en utilisant le model Bart

def encode_data_bart(texts, summaries):
    # Tokenisation par lots
    inputs = tokenizer_Bart(texts, padding=True, truncation=True, return_tensors="tf", max_length=1024)
    
    # Convertir les résumés en liste si c'est un tensor
    if isinstance(summaries, tf.Tensor):
        summaries = summaries.numpy().tolist()

    # Vérification que tous les résumés sont des chaînes
    if not all(isinstance(summary, str) for summary in summaries):
        raise ValueError("Tous les éléments de 'summaries' doivent être des chaînes.")

    # Créer des labels : 1 si la phrase est dans le résumé, 0 sinon
    labels = [1 if any(sentence in summary for summary in summaries) else 0 for sentence in texts]

    return inputs['input_ids'], inputs['attention_mask'], tf.convert_to_tensor(labels)  

In [ ]:
# Encoder les données d'entraînement
X_train_input_ids, X_train_attention_mask, y_train_labels = encode_data_bart(X_train, y_train)

In [ ]:
# Encoder les données de validation
X_val_input_ids, X_val_attention_mask, y_val_labels = encode_data_bart(X_val, y_val)

In [ ]:
# Fonction pour créer un Dataset TensorFlow
def create_tf_dataset(texts, summaries):
    input_ids, attention_mask, labels = encode_data_bart(texts, summaries)
    
    # Créer un Dataset TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels))
    
    # Prétraiter les données pour un chargement efficace
    dataset = dataset.shuffle(1000).batch(16).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [ ]:
# Encodage et conversion de X_train et X_val en objets Dataset
train_dataset = create_tf_dataset(X_train, y_train)
val_dataset = create_tf_dataset(X_val, y_val)

In [ ]:
# Affichage des informations sur les Datasets
print("Train Dataset:", train_dataset)
print("Validation Dataset:", val_dataset)

In [ ]:
# Création des objets Dataset de TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_input_ids, X_train_attention_mask, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_input_ids, X_val_attention_mask, y_val))

In [ ]:
# Prétraitement des données pour un chargement optimal
train_dataset = train_dataset.shuffle(1000).batch(8).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(8).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Compilation du modèle avec une fonction de perte et un optimiseur

model_Bart.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5), loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# Entraînement du modèle
model_Bart.fit(train_dataset, validation_data=val_dataset, epochs=3)

In [ ]:
#Validation du Modèle

metric = evaluate.load('glue', 'mrpc')
# Example of typical usage
for batch in val_dataset:
    inputs, references = batch
    predictions = model_Bart(inputs)
    metric.add_batch(predictions=predictions, references=references)
score = metric.compute()

## 4. Évaluation des performances

Génération de résumés :

Générer des résumés à partir du jeu de test pour l’approche abstraite.

Comparaison des approches :

Évaluer les résumés générés (abstrait et extractif) en utilisant les métriques ROUGE et BLEU.
Comparer les scores et analyser les résultats.


In [ ]:
#Exemple de generation de resume de X_test_totext
input_text = X_test_totext
inputs = tokenizer_Bart(input_text, return_tensors='tf')
outputs = model_Bart.generate(inputs['input_ids'])
generated_text = tokenizer_Bart.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [ ]:

#Exemple de generation de resume d'un article https://lefaso.net/spip.php?article134735
input_text = "Le Burkina Faso a réalisé une évaluation initiale sur le mercure en 2018, qui a montré un apport total annuel de mercure estimé est à 353 590 kg. De plus, un inventaire effectué de 2016 à 2021 dans 348 établissements de santé et a montré une quantité de 126 492 thermomètres à mercure et 8 973 sphygmomanomètres à mercure. L’étude menée en 2022 sur l’état actuel de mise en œuvre de la Convention de Minamata sur le mercure a montré que le Burkina Faso a importé 15 kg de thermomètres à mercure. Les sphygmomanomètres à mercure sont utilisés pour mesurer la tension artérielle et les thermomètres à mercure pour mesurer la température. Pourtant, selon le représentant pays par intérim de l’OMS, représenté par Dr Ramatou Sawadogo, le mercure peut avoir des effets toxiques sur les systèmes nerveux, digestif et immunitaire, ainsi que sur les poumons, les reins, la peau et les yeux. Pour lui, il n’existe pas de niveau d’exposition au mercure élémentaire qui soit sans risque pour le corps humain ; des effets peuvent être constatés même à de très faibles concentrations. « Les fœtus, les nouveau-nés et les enfants sont parmi les plus vulnérables et les plus sensibles aux effets nocifs du mercure », a-t-il déclaré dans son discours lu par Dr Ramatou Sawadogo."
inputs = tokenizer_Bart(input_text, return_tensors='tf')
outputs = model_Bart.generate(inputs['input_ids'])
generated_text = tokenizer_Bart.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

## 5. Expérimentations additionnelles (option)

### Tests sur d'autres sources :

Tester les modèles sur des textes provenant d'autres sources pour évaluer la généralisation.

### Variantes du modèle :

Expérimenter avec différents modèles pré-entraînés (GPT) et différentes configurations des architectures (taille des couches, nombre d'époques).

### Exploration de techniques avancées :

Tester des techniques comme la régularisation ou l'ajout de contraintes pour améliorer la qualité des résumés.


In [ ]:
article = pd.read_csv('C:/Users/pc/Desktop/TP/NLP2/TP2/article resume.txt')
resum_ref = pd.read_csv('C:/Users/pc/Desktop/TP/NLP2/TP2/resume reference.txt')

In [ ]:
# Nettoyer le text de l'article
cleaned_new_texts = [clean_text(text) for text in article]

In [ ]:
# Exemple avec BART
def generate_summary(text):
    inputs = tokenizer_bart(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model_bart.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    return tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)

# Générer des résumés pour les nouveaux textes
generated_summaries = [generate_summary(text) for text in cleaned_new_texts]
print("Résumés générés :", generated_summaries)

In [ ]:
# Évaluation des Résumés

# Charger les métriques ROUGE et BLEU
rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('bleu')

# Calculer les scores ROUGE et BLEU
rouge_scores = rouge_metric.compute(predictions=generated_summaries, references=resum_ref)
bleu_scores = bleu_metric.compute(predictions=generated_summaries, references=[resum_ref])

print("Scores ROUGE :", rouge_scores)
print("Scores BLEU :", bleu_scores)

## 6. Questions de réflexion (voir le rapport pour plus de details)

### Analyse des limites :
Quelles sont les limites des modèles pour resumé extractifs par rapport aux modèles pour resumé abstraits ?
Quelles sont les limites des modèles encoder-decoder dans les tâches de résumé de texte ?
Comment améliorer la performance du modèle sur des textes complexes ou longs ?

## 7. Ressources
Dataset CNN/Daily Mail pour le résumé de texte
Bibliothèques recommandées :
Hugging Face Transformers pour les modèles pré-entraînés.
Scikit-learn pour les évaluations et la préparation des données.
NLTK ou SpaCy pour le traitement du langage naturel.

## Livrables

### Dépôt GitHub :

Créer un dépôt contenant le code, les notebooks d’analyse, et un fichier README expliquant comment exploiter votre code.

### Rapport final :

Inclure une analyse comparative des performances, des graphiques et des réflexions sur les deux approches.

### Images & Captures d'écran :

Ajouter des images et captures d'écran des résultats des évaluations, y compris les métriques ROUGE et BLEU.
